# BestBuy Use Case
Elvis

# Loading Packages

In [1]:
from selenium.common.exceptions import NoSuchElementException
import os
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import pandas as pd
import re
from selenium.webdriver.common.keys import Keys
import time
import datetime
from IPython.display import clear_output

In [2]:
# selenium 4
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

google = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
google.implicitly_wait(15)

# search page


In [3]:
google.get('https://www.bestbuy.com/#?intl=nosplash')
choose_country_page = google.current_url
while choose_country_page == google.current_url:
    try:
        google.find_element(By.XPATH, '/html/body/div[2]/div/div/div/div[1]/div[2]/a[2]').click()
    except:
        pass
google.refresh()

In [4]:
google.find_element(By.XPATH, '//*[@id="gh-search-input"]').click()
google.find_element(By.XPATH, '//*[@id="gh-search-input"]').send_keys('ASUS')
google.find_element(By.XPATH, '//*[@id="gh-search-input"]').send_keys(Keys.ENTER)

In [5]:
html = google.find_element(By.TAG_NAME, 'html')
html.send_keys(Keys.END)
time.sleep(5)
html.send_keys(Keys.HOME)
time.sleep(3)
sku_items = google.find_elements(By.XPATH, '//*[@id="main-results"]/ol/li/div/div/div[not(contains(@class, "shop-display-ad"))]')

In [23]:
# 创建一个空的DataFrame，用于存储结果
# result_df = pd.DataFrame(columns=["Titles", "Product_URL", "Models", "Sku", "Star", "Reviews_num", "RR_URL", "Prices"])
result_df = pd.DataFrame(columns=["Titles", "Product_URL", "Models", "Sku", "StarNum", "ReviewsNum", "Prices", 'Save', 'Old Price', "Button Status"])

rr_pattern = r'Rating (\d+\.\d+) out of 5 stars with (\d+) reviews'
cnt = -1

# 循环处理每个sku_items中的元素
for sku in sku_items:
    cnt += 1
    clear_output(wait=True)
    print(f"{cnt}/{len(sku_items)}")

    # 向下滚动页面
    html = google.find_element(By.TAG_NAME, 'html')
    html.send_keys(Keys.PAGE_DOWN)

    try:
        # 获取商品标题
        title = sku.find_element(By.XPATH, ".//h4[@class='sku-title']").get_attribute('textContent')

        # 获取产品链接
        prod_url = sku.find_element(By.XPATH, ".//h4[@class='sku-title']/a").get_attribute('href')

        # 获取型号和SKU
        model = sku.find_elements(By.XPATH, ".//span[@class='sku-value']")[0].text
        SKUdigit = sku.find_elements(By.XPATH, ".//span[@class='sku-value']")[1].text

        # 获取星级和评论数
        rr = sku.find_element(By.XPATH, ".//div[@class='ratings-reviews']//p[@class='visually-hidden']").text
        match = re.search(rr_pattern, rr)
        if match:
            star = float(match.group(1))
            reviews_num = int(match.group(2))
        else:
            star = None
            reviews_num = None

        # 获取评论链接
        #rr_url = sku.find_element(By.XPATH, ".//div[@class='ratings-reviews']/a").get_attribute('href')

        # 获取价格
        price = sku.find_element(By.XPATH, ".//div[@class='sku-list-item-price']//span[@aria-hidden='true']").text
        button_status = sku.find_element(By.XPATH, ".//div[@class='sli-add-to-cart']//button").text
        try:
            save = sku.find_element(By.XPATH,".//div[@class='sku-list-item-price']//div[@data-testid='savings-regular-price']/div[1]").text
            old_price = sku.find_element(By.XPATH,".//div[@class='sku-list-item-price']//div[@data-testid='savings-regular-price']/div[2]").get_attribute('textContent')
        except NoSuchElementException:
            save = None
            old_price = None
        
    except NoSuchElementException:
        # 如果出现NoSuchElementException，说明某些元素未找到，可以选择跳过或采取其他操作
        pass
    # 创建一行数据
    row = [title, prod_url, model, SKUdigit, star, reviews_num, price, save, old_price, button_status]
    # 将一行数据添加到DataFrame中
    result_df.loc[cnt] = row
    if cnt == 5:
        break
collect_date = pd.DataFrame({"date": [datetime.date.today()] * result_df.shape[0]})
# Concatenate xxx and result_df along columns axis (axis=1)
result_df = pd.concat([collect_date, result_df], axis=1)


5/21


In [32]:
result_df

,date,Titles,Product_URL,Models,Sku,StarNum,ReviewsNum,Prices,Save,Old Price,Button Status
0,2023-12-07,"ASUS - Zenbook 14X 14.5"" 2.8K OLED Touch Lapto...",https://www.bestbuy.com/site/asus-zenbook-14x-...,Q420VA-EVO.I7512,6543526,4.6,475,$699.99,Save $300,Was $999.99The previous price was $999.99,Add to Cart
1,2023-12-07,"ASUS - Zenbook 14X 14.5"" 2.8K OLED Touch Lapto...",https://www.bestbuy.com/site/asus-zenbook-14x-...,Q420VA-EVO.I7512,6543526,4.6,475,$699.99,Save $300,Was $999.99The previous price was $999.99,Add to Cart
2,2023-12-07,"ASUS - 15.6"" OLED Laptop - Intel Core i9-13900...",https://www.bestbuy.com/site/asus-15-6-oled-la...,Q540VJ-I93050,6534578,4.6,421,"$1,299.99",None,None,Add to Cart
3,2023-12-07,"ASUS - TUF Gaming A16 16"" 165Hz Gaming Laptop ...",https://www.bestbuy.com/site/asus-tuf-gaming-a...,FA617NS-A16.R77600,6535499,4.5,445,$729.99,Save $370,"Was $1,099.99The previous price was $1,099.99",Add to Cart
4,2023-12-07,ASUS - ROG Zephyrus G14 14” 165Hz Gaming Lapto...,https://www.bestbuy.com/site/asus-rog-zephyrus...,GA402XV-G14.R94060,6535495,4.5,385,"$1,299.99",Save $300,"Was $1,599.99The previous price was $1,599.99",Add to Cart
5,2023-12-07,"ASUS - TUF 15.6"" Gaming Laptop - Intel Core i7...",https://www.bestbuy.com/site/asus-tuf-15-6-gam...,FX507ZI-F15.I74070,6535503,4.7,246,"$1,399.99",None,None,Add to Cart


In [33]:
rr_df = pd.DataFrame(columns=['who', 'time', 'star', 'subject', 'cmt', 'helpful', 'unhelpful'])
cnt = -1
for url in result_df.Product_URL:
    google.get(url)
    google.find_element(By.XPATH, '//*[@class="see-all-reviews-button-container"]/a[1]').click()
    while True:
        review_items = google.find_elements(By.XPATH, '//*[@id="reviews-accordion"]/div[1]/ul/li')
        for rr in review_items:
            clear_output(wait=True)
            print(cnt)
            try:
                cnt += 1
                who = rr.find_element(By.XPATH, './div/div/div').text # name
                star = rr.find_element(By.XPATH, './div/div[2]/div[1]/div/div/p').get_attribute('textContent') #star
                time = rr.find_element(By.XPATH, './div/div[2]/div[3]/div[2]/time').get_attribute('title') # submission time
                cmt = rr.find_element(By.XPATH, './div/div[2]/div[5]/p').text # comment
                subject = rr.find_element(By.XPATH, './div/div[2]/div[1]/h4').get_attribute('textContent') # subject
                helpful = rr.find_element(By.XPATH, './div/div[2]/div[7]/div/div/div[1]/button[1]').get_attribute('textContent') # helpful
                unhelpful = rr.find_element(By.XPATH, './div/div[2]/div[7]/div/div/div[1]/button[2]').get_attribute('textContent') # unhelpful
                row = [who, time, star, subject, cmt, helpful, unhelpful]
                rr_df.loc[cnt] = row
            except:
                print('pass')
                pass
        try:
            google.find_element(By.XPATH, '//*[@id="reviews-accordion"]/div[2]/div/div[2]/ul/li[9]/a').click()
        except:
            break
collect_date = pd.DataFrame({"date": [datetime.date.today()] * rr_df.shape[0]})
# Concatenate xxx and result_df along columns axis (axis=1)
rr_df = pd.concat([collect_date, rr_df], axis=1)

995


In [1]:
rr_df

NameError: name 'rr_df' is not defined

# brand

In [8]:
dellurl = 'https://www.bestbuy.com/site/brands/dell/pcmcat140500050010.c?id=pcmcat140500050010'
google.get(dellurl)

In [9]:
google.find_element(By.XPATH, '//*[@id="widget-d219661a-faa3-4ed3-adbf-9ba1853e1e88"]/div/div[2]/div/div[1]/div/a').click()

In [10]:
html = google.find_element(By.TAG_NAME, 'html')
html.send_keys(Keys.END)
time.sleep(5)
html.send_keys(Keys.HOME)
time.sleep(3)
brand_res_list = google.find_elements(By.XPATH, '//*[@id="main-results"]/ol/li/div/div/div[not(contains(@class, "shop-display-ad"))]')

In [11]:
# 创建一个空的DataFrame，用于存储结果
brand_result_df = pd.DataFrame(columns=["Titles", "Product_URL", "Models", "Sku", "Star", "Reviews_num", "RR_URL", "Prices", "Save", "Previous Prices", 'AddtoCart button'])

rr_pattern = r'Rating (\d+\.\d+) out of 5 stars with (\d+) reviews'
cnt = -1

# 循环处理每个sku_items中的元素
for sku in brand_res_list:
    cnt += 1
    print(cnt)

    # 向下滚动页面
    html = google.find_element(By.TAG_NAME, 'html')
    html.send_keys(Keys.PAGE_DOWN)

    
    
    try:
        # 获取商品标题
        title = sku.find_element(By.XPATH, ".//h4[@class='sku-title']").get_attribute('textContent')
        # 获取产品链接
        prod_url = sku.find_element(By.XPATH, ".//h4[@class='sku-title']/a").get_attribute('href')
        # 获取型号和SKU
        model = sku.find_elements(By.XPATH, ".//span[@class='sku-value']")[0].text
        SKUdigit = sku.find_elements(By.XPATH, ".//span[@class='sku-value']")[1].text
        # 获取评论链接
        rr_url = sku.find_element(By.XPATH, ".//div[@class='ratings-reviews']/a").get_attribute('href')
        # 获取价格
        price = sku.find_element(By.XPATH, ".//div[@class='sku-list-item-price']//span[@aria-hidden='true']").text
    except NoSuchElementException:
        continue

    # 获取星级和评论数
    rr = sku.find_element(By.XPATH, ".//div[@class='ratings-reviews']//p[@class='visually-hidden']").text
    match = re.search(rr_pattern, rr)
    if match:
        star = float(match.group(1))
        reviews_num = int(match.group(2))
    else:
        star = None
        reviews_num = None
        
    # save and old price
    try:
        save = sku.find_element(By.XPATH, ".//div[@class='pricing-price__savings pricing-price__savings--promo-red']").text
        old_price = sku.find_element(By.XPATH, './/div[@class="pricing-price__regular-price-content"]/div[1]').text
    except NoSuchElementException:
        save = 0
        old_price = price

    # addtocart status
    try:
        addtocart = sku.find_element(By.XPATH, ".//div[@class='sli-add-to-cart']//button").get_attribute('textContent')
    except NoSuchElementException:
        addtocart = 'Nothing'

    # 创建一行数据
    row = [title, prod_url, model, SKUdigit, star, reviews_num, rr_url, price, save, old_price, addtocart]

    # 将一行数据添加到DataFrame中
    brand_result_df.loc[cnt] = row




0
1


C:\Users\ElvisBY_Chen\AppData\Local\Temp\ipykernel_22256\4277853596.py:61: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  brand_result_df.loc[cnt] = row


2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [12]:
brand_result_df

,Titles,Product_URL,Models,Sku,Star,Reviews_num,RR_URL,Prices,Save,Previous Prices,AddtoCart button
0,"Dell - Inspiron 3420 14"" Touch Laptop - Qualco...",https://www.bestbuy.com/site/dell-inspiron-342...,i3420-S476SLV-PUS,6525656,4.3,68,https://www.bestbuy.com/site/dell-inspiron-342...,$549.99,0,$549.99,Add to Cart
1,Dell - Inspiron 15 3535 Touch Laptop - AMD Ryz...,https://www.bestbuy.com/site/dell-inspiron-15-...,i3535-A766BLK-PUS,6559387,NaN,None,https://www.bestbuy.com/site/dell-inspiron-15-...,$329.99,Save $270,Was $599.99,Add to Cart
2,"Dell - Inspiron 16.0"" 2-in-1 Touch Laptop -13t...",https://www.bestbuy.com/site/dell-inspiron-16-...,i7630-7312SLV-PUS,6539902,4.7,323,https://www.bestbuy.com/site/dell-inspiron-16-...,$799.99,Save $300,"Was $1,099.99",Add to Cart
3,"Dell - Inspiron 14.0"" 2-in-1 Touch Laptop - 13...",https://www.bestbuy.com/site/dell-inspiron-14-...,i7430-7374SLV-PUS,6539910,4.7,515,https://www.bestbuy.com/site/dell-inspiron-14-...,$799.99,Save $200,Was $999.99,Add to Cart
4,"Dell - Inspiron 14.0"" 2-in-1 Touch Laptop - AM...",https://www.bestbuy.com/site/dell-inspiron-14-...,i7435-A111BLU-PUS,6538395,4.6,396,https://www.bestbuy.com/site/dell-inspiron-14-...,$499.99,Save $200,Was $699.99,Add to Cart
7,"Dell - Inspiron 16.0"" 2-in-1 Touch Laptop - 13...",https://www.bestbuy.com/site/dell-inspiron-16-...,i7630-5640SLV-PUS,6539908,4.8,98,https://www.bestbuy.com/site/dell-inspiron-16-...,$599.99,Save $300,Was $899.99,Add to Cart
8,"Dell - XPS 15 15.6"" 3.5K OLED Touch-Screen Lap...",https://www.bestbuy.com/site/dell-xps-15-15-6-...,XPS9530-9565SLV-PUS,6540609,4.8,50,https://www.bestbuy.com/site/dell-xps-15-15-6-...,"$2,799.99",0,"$2,799.99",Add to Cart
9,"Dell - Inspiron 16.0"" 2-in-1 OLED Touch Laptop...",https://www.bestbuy.com/site/dell-inspiron-16-...,i7630-7305BLU-PUS,6539903,4.6,76,https://www.bestbuy.com/site/dell-inspiron-16-...,"$1,499.99",0,"$1,499.99",Add to Cart
10,"Dell - XPS 15 15.6"" FHD+ Laptop - Intel Core i...",https://www.bestbuy.com/site/dell-xps-15-15-6-...,XPS9530-7701SLV-PUS,6540610,4.7,89,https://www.bestbuy.com/site/dell-xps-15-15-6-...,"$1,999.99",0,"$1,999.99",Add to Cart
11,"Dell XPS 13 13.4"" FHD+ Laptop - 12th Gen Intel...",https://www.bestbuy.com/site/dell-xps-13-13-4-...,XPS9315-7725SKY-PUS,6540612,4.4,140,https://www.bestbuy.com/site/dell-xps-13-13-4-...,"$1,349.99",0,"$1,349.99",Add to Cart
